In [2]:
import numpy as np

def  edit_distance(x, y, D):
    for j in range(len(y)):
        D[0][j] = j  #对应edit_distance(0, j) = j
    for i in range(len(x)):
        D[i][0] = i  #对应edit_distance(i, 0) = i
    for i in range(1, len(x)):
        for j in range(1, len(y)):  #根据递归表达式D[i][j]的值由D[i-1][j],D[i][j-1],D[i-1][j-1]决定
            diff_xy = 0
            if x[i] != y[j]:
                diff_xy = 1
            D[i][j] = min(1 + D[i-1][j], 1 + D[i][j-1], diff_xy + D[i-1][j-1])
    return D
x = "POLYNOMIAL"
y = "EXPONENTIAL"
D = np.zeros((len(x), len(y)))
D = edit_distance(x, y, D)
print("edit distance table is:\n", D)
print("the smallest edit distance between {0} and {1} is {2}".format(x, y, D[len(x)-1][len(y) - 1]))

edit distance table is:
 [[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  1.  2.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 2.  2.  2.  3.  3.  4.  5.  6.  7.  8.  8.]
 [ 3.  3.  3.  3.  4.  4.  5.  6.  7.  8.  9.]
 [ 4.  4.  4.  4.  3.  4.  4.  5.  6.  7.  8.]
 [ 5.  5.  5.  4.  4.  4.  5.  5.  6.  7.  8.]
 [ 6.  6.  6.  5.  5.  5.  5.  6.  6.  7.  8.]
 [ 7.  7.  7.  6.  6.  6.  6.  6.  6.  7.  8.]
 [ 8.  8.  8.  7.  7.  7.  7.  7.  7.  6.  7.]
 [ 9.  9.  9.  8.  8.  8.  8.  8.  8.  7.  6.]]
the smallest edit distance between POLYNOMIAL and EXPONENTIAL is 6.0


In [3]:
import sys
N = 4 #相乘的矩阵数
M = []
M.append((50, 20))  #构造5.2.1节对应的4个矩阵维度
M.append((20, 1))
M.append((1, 10))
M.append((10, 100))
arrange_map = {} #记录给定i和j对应的最优分割k
C = np.zeros((N, N)).astype(int)
for i in range(N):
    for j in range(N):
        C[i][j] = -1
def  compute_c_table(C, i, j):
    if i < 0 or i >= N:
        return
    if j < 0 or j >= N:
        return
    if i == j:
        C[i][j] = 0
    if C[i][j] != -1:  #当前元素已经计算过
        return C[i][j]
    cost = sys.maxsize 
    best_k = i
    for k in range(i, j):
        first_part_cost = compute_c_table(C, i, k)
        second_part_cost = compute_c_table(C, k + 1, j)
        if first_part_cost + second_part_cost + M[i][0]*M[k][1]*M[j][1] < cost:
            cost = first_part_cost + second_part_cost + M[i][0]*M[k][1]*M[j][1]
            best_k = k 
    C[i][j] = int(cost) 
    arrange_map[(i, j)] = best_k
    return C[i][j]
compute_c_table(C, 0, N-1)
print(C)
print("the best computation time is: ", C[0][N-1])
def  print_best_matrix_arrange(arrange_map, i, j):
    if i == j:
        return "A{0}".format(i)
    if j == i + 1:
        return "(A{0}*A{1})".format(i, j)
    k = arrange_map[(i, j)]  #得到矩阵Ai到Aj连乘时的最优分割
    left = print_best_matrix_arrange(arrange_map, i, k)
    right = print_best_matrix_arrange(arrange_map, k+1, j)
    return "({0}*{1})".format(left, right)
arrangement = print_best_matrix_arrange(arrange_map, 0, N-1)
print("the best arrangement is :", arrangement)

[[   0 1000 1500 7000]
 [  -1    0  200 3000]
 [  -1   -1    0 1000]
 [  -1   -1   -1    0]]
the best computation time is:  7000
the best arrangement is : ((A0*A1)*(A2*A3))


In [4]:
N = 5  #节点数
D = np.zeros((N, N)).astype(int)
S = set()  #对应点的集合
for i in range(1, N):
    S.add(i)
D[0][0] = 0  #根据图5-5进行初始化
D[0][1] = 2
D[0][2] = 4
D[0][3] = 1
D[0][4] = 2
D[1][0] = 3
D[1][1] = 0
D[1][2] = 3
D[1][3] = 2
D[1][4] = 3
D[2][0] = 4
D[2][1] = 3
D[2][2] = 0
D[2][3] = 4
D[2][4] = 2
D[3][0] = 1
D[3][1] = 2
D[3][2] = 4
D[3][3] = 0
D[3][4] = 2
D[4][0] = 2
D[4][1] = 3
D[4][2] = 2
D[4][3] = 2
D[4][4] = 0
def  get_subset(point_set, n): #根据点数构造子集
    if n == 0:
        return []
    sub_sets = [] 
    for p in point_set:
        s = set()
        s.add(p)
        set_copy = point_set.copy()
        set_copy.discard(p)
        sets = get_subset(set_copy, n - 1)
        if len(sets) == 0:
            sub_sets.append(s)
        else:
            for each_set in sets:
                s_union = s.copy().union(each_set)
                sub_sets.append(s_union)
    return sub_sets
def  find_shortest_path():  #计算最短路径
    C = {}
    for point_count in range(1, N):
        sub_sets = get_subset(S.copy(), point_count)  #获得包含给定点数的子集
        for the_set in sub_sets:
            distances = {}  #记录起始点到集合内每一点的最短距离
            for the_point in the_set: #计算当前集合内的点作为回到起始点前一个点时对应的最短距离
                after_discard = the_set.copy()
                after_discard.discard(the_point)
                if len(after_discard) == 0:
                    distances[the_point] = D[0][the_point]
                else:
                    '''
                    如果集合S包含三个点{1,2,3},从节点0开始遍历集合中所有点的最短路径算法为,先找出从起始点0开始遍历集合{1,2},{1,3},{2,3}的最短路径，
                    然后用这三条路径长度分别加上D[2][3],D[3][2],D[3][1]，三个结果中的最小值就是从起始点0开始，遍历集合{1,2,3}中所有点后的最短路径.
                    因此当集合为{1,2,3}时，C[{1,2,3}]对应key值为1，2，3的一个map对象，map[1]表示集合点为{1,2,3}时，遍历路径最后一个节点时1时的最短距离
                    '''
                    set_copy = the_set.copy()
                    set_copy.discard(the_point)
                    distance_to__points = C[frozenset(set_copy)]
                    d = sys.maxsize 
                    for p in set_copy:
                        if D[p][the_point] + distance_to__points[p] < d:
                            d = D[p][the_point] + distance_to__points[p] 
                    distances[the_point] = d
            C[frozenset(the_set)] = distances.copy() #记录起始点到当前集合内每个点的最短距离,根据python语法,fronzenset才能作为key
            distances.clear()
    distances = C[frozenset(S)]
    d = sys.maxsize
    for point in S: #先找到起始点到集合{1,2...n}中对应点的嘴短距离
        if distances[point] + D[point][0] < d:
            d = distances[point] + D[point][0]
    return d, C
shortest_distance, C = find_shortest_path()
print("the shortest distance for visiting all points is : ", shortest_distance)


the shortest distance for visiting all points is :  10


In [5]:
def  find_visiting_path(point_set, C):
    if len(point_set)== 1:
        return [point_set.pop()]
    distances = C[frozenset(point_set)]
    d = sys.maxsize
    path_point = []
    selected_point = None
    for point in S: #先找到起始点到集合{1,2...n}中对应点的嘴短距离
        if distances[point] + D[point][0] < d:
            d = distances[point] + D[point][0]
            selected_point = point 
    point_set.discard(selected_point)
    points_before = find_visiting_path(point_set, C)
    path_point.extend(points_before)
    path_point.extend([selected_point])
    return path_point
points =    find_visiting_path(S, C)
print("the shortest path for visiting all points is : ", points) 

the shortest path for visiting all points is :  [1, 2, 4, 3]


In [6]:
D = np.zeros((3, 3))
D[0][0] = 0.7 #初始化状态转换表
D[0][1] = 0.2
D[0][2] = 0.1
D[1][0] = 0.4
D[1][1] = 0.4
D[1][2] = 0.2
D[2][0] = 0.2
D[2][1] = 0.5
D[2][2] = 0.3
def  viterbi(n):  #计算第一天个状态开始，经过n天后抵达各状态的最大概率
    C = np.zeros((3, 3, n + 1))
    for i in range(3):
        C[i][i][0] = 1
    for k in range(1, n + 1): #根据公式(3)迭代最大概率路径
        for i in range(3):
            for j in range(3):
                C[i][j][k] = max(C[i][0][k - 1] * D[0][j], C[i][1][k-1] * D[1][j], C[i][2][k - 1] * D[2][j])
    return C
n = 4
C = viterbi(n)       
for i in range(3):
    for j in range(3):
        print("the biggest probability from status {0} to status{1} after {2} days is {3}: ".format(i, j, n, C[i][j][n-1]))
def  find_probability_path(i, j , k):  #获得第一天从状态i开始经过k天后抵达状态j的最大概率路径
    path = []
    while k > 0:
        t = 0
        p = C[i][0][k - 1] * D[0][j]
        if C[i][1][k - 1] * D[1][j] > p:
            p = C[i][1][k - 1] * D[1][j]
            t = 1
        if C[i][2][k - 1] * D[1][j] > p:
            p = C[i][1][k - 1] * D[1][j]
            t = 1
        path.append(t)
        k -= 1
    path.reverse()
    path.append(j)
    path.insert(0, i)
    return path
days = 3
i = 2
j = 2
path = find_probability_path(i, j, days - 1)
print("the biggest probability path from status {0} to status{1} after {2} days is : {3}".format(i, j, days, path))

the biggest probability from status 0 to status0 after 4 days is 0.3429999999999999: 
the biggest probability from status 0 to status1 after 4 days is 0.09799999999999999: 
the biggest probability from status 0 to status2 after 4 days is 0.048999999999999995: 
the biggest probability from status 1 to status0 after 4 days is 0.19599999999999998: 
the biggest probability from status 1 to status1 after 4 days is 0.06400000000000002: 
the biggest probability from status 1 to status2 after 4 days is 0.03200000000000001: 
the biggest probability from status 2 to status0 after 4 days is 0.13999999999999999: 
the biggest probability from status 2 to status1 after 4 days is 0.08000000000000002: 
the biggest probability from status 2 to status2 after 4 days is 0.04000000000000001: 
the biggest probability path from status 2 to status2 after 3 days is : [2, 1, 1, 2]
